In [1]:
!pip install -q parlai
!pip install wandb

     |████████████████████████████████| 1.3MB 4.3MB/s 
     |████████████████████████████████| 163kB 56.9MB/s 
     |████████████████████████████████| 40kB 6.9MB/s 
     |████████████████████████████████| 552kB 53.8MB/s 
     |████████████████████████████████| 133kB 60.4MB/s 
     |████████████████████████████████| 7.5MB 57.4MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 133kB 62.4MB/s 
     |████████████████████████████████| 245kB 51.5MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 3.3MB 53.4MB/s 
     |████████████████████████████████| 133kB 59.6MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 2.7MB 51.2MB/s 
     |████████████████████████████████| 122kB 59.4MB/s 
     |████████████████████████████████| 215kB 58.0MB/s 
     

In [3]:
!nvidia-smi

Mon May  3 03:28:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
 !pip install colab_ssh --upgrade
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="KG8VkPp2GyI6781GWg")

In [5]:
working_dir = "/content/drive/MyDrive/Dev/ECEN689/ECEN689"
print(working_dir)
from parlai.scripts.train_model import TrainModel
import os 
import numpy as np
import pandas as pd

/content/drive/MyDrive/Dev/ECEN689/ECEN689


In [ ]:
# train a fine tune model
# TrainModel.main(
#     # similar to before
#     task='empathetic_dialogues', 
#     model='transformer/generator',
#     model_file='from_pretrained/model',
    
#     # initialize with a pretrained model
#     init_model='zoo:tutorial_transformer_generator/model',
    
#     # arguments we get from the pretrained model.
#     # Unfortunately, these must be looked up separately for each model.
#     n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
#     label_truncate=128, ffn_size=2048, embedding_size=512,
#     activation='gelu', variant='xlm',
#     dict_lower=True, dict_tokenizer='bpe',
#     dict_file='zoo:tutorial_transformer_generator/model.dict',
#     learn_positional_embeddings=True,
    
#     # some training arguments, specific to this fine-tuning
#     # use a small learning rate with ADAM optimizer
#     lr=1e-5, optimizer='adam',
#     warmup_updates=100,
#     # early stopping on perplexity
#     validation_metric='ppl',
#     # train at most 10 minutes, and validate every 0.25 epochs
#     max_train_time=600, validation_every_n_epochs=0.25,
    
#     # depend on your gpu. If you have a V100, this is good
#     batchsize=12, fp16=True, fp16_impl='mem_efficient',
    
#     # speeds up validation
#     skip_generation=True,
    
#     # helps us cram more examples into our gpu at a time
#     dynamic_batching='full',
#     verbose=False,
#     wandb_log=True,
#     wandb_project="ecen689-project"
# )

In [ ]:
# Import the Interactive script
from parlai.scripts.interactive import Interactive

# call it with particular args
Interactive.main(
    model_file=os.path.join(working_dir, "from_pretrained/model"),
    # model_file='zoo:dodecadialogue/empathetic_dialogues_ft/model',
    inference="beam", 
    beam_size=5, beam_min_length=10, beam_block_ngram=3, beam_context_block_ngram=3
)

In [14]:
import numpy as np
import copy
import signal
import json
from os.path import isfile


In [17]:
import torch

from torch.autograd import backward

from parlai.core.worlds import MultiAgentDialogWorld
from collections import namedtuple
from itertools import chain

Action = namedtuple('Action', ['id', 'action', 'responses'])
"""
A node in the action-response tree.
:param id: str, the id of the actor.
:param action: dict, containing the agent actions.
:param responses: dict, containing the agent's response for the action.
"""

ACTIVE, STATIC = 'active', 'static'

DULL_RESPONSES = [
    {'labels': "i do n ' t know ."},
    {'labels': 'why ?'},
    {'labels': 'i am sorry .'},
    {'labels': 'i see .'}
]

def calculate_dull_log_prob(resp_batch, agent):
    model = agent.model
    model.eval()
    print(resp_batch)
    batch = agent.batchify([
        {**resp_batch, 
        'labels': dull_response['labels'],
        'labels_vec': agent._vectorize_text(dull_response['labels'])} for
        dull_response in DULL_RESPONSES
    ])
    out = model(batch.text_vec, ys=batch.label_vec)
    # generated response

    notnull = batch.label_vec.ne(agent.NULL_IDX)
    target_tokens = notnull.long().sum().item()

    scores = out[0]
    score_view = scores.view(-1, scores.size(-1))
    loss = agent.criterion(score_view, batch.label_vec.view(-1))
    loss = loss.sum().data.cpu().numpy() / target_tokens / len(DULL_RESPONSES)

    return loss

class RLDialogWorld(MultiAgentDialogWorld):

    def __init__(self, opt, active_agent, static_agent, 
                 teacher, shared=None):
        """"""
        self.id = 'RLDialogWorld'
        self.episode_batch = None
        self.active_agent = active_agent
        self.static_agent = static_agent
        agents = teacher + [static_agent, active_agent]

        super(RLDialogWorld, self).__init__(opt, agents, shared)

    def parley(self):
        """"""
        # Initial sentence from the dataset
        initial_action = self.agents[0].act()

        self.active_agent.zero_grad()
        actions = self.rollout(initial_action)

        with torch.no_grad():
            reward = self.calculate_reward(actions, 1)

        self.active_agent.observe({'reward': reward})
        self.active_agent.update_params()

    def iterate_reponses(self, action):
        """"""
        for response in action.responses:
            yield {
                'text': action.action['text'], 
                'text_vec': action.action['text_vec'].cpu(), 
                'labels': response.action['text'],
                'labels_vec': response.action.get('text_vec', 
                    self.static_agent._vectorize_text(
                        response.action['text']).cpu())
            }

    def calculate_reward(self, action, weight):
        """"""
        reward = 0

        if len(action.responses) == 0:
            return 0

        if action.id == ACTIVE:
            for resp in self.iterate_reponses(action):
                batch = self.active_agent.batchify([resp])

                log_prob = self.static_agent.compute_log_prob(batch, False)

                reward += log_prob - calculate_dull_log_prob(
                    resp, self.static_agent)

        # Reduce reward significance for next dialogue turns
        weight = weight * self.opt['reward_decay']
        for response in action.responses:
            reward += self.calculate_reward(response, weight)
           
        return reward
    
    def rollout(self, initial_action):
        """"""
        def roll(action, num_rollouts):
            """"""
            if action.id == ACTIVE:
                self.static_agent.observe(action.action)
                act = self.static_agent.act()
                act['text_vec'] = act['text_vec'].cpu()
                static_action = Action(
                    id=STATIC, 
                    action=act, 
                    responses=[])

                action.responses.append(
                    roll(static_action, num_rollouts))

            else:
                if num_rollouts == 0:
                    return action

                self.active_agent.observe(action.action)
                for _ in range(self.opt['dialog_branches']):
                    act = self.active_agent.act()
                    act['text_vec'] = act['text_vec'].cpu()
                    active_action = Action(
                        id=ACTIVE,
                        action=act, 
                        responses=[])

                    action.responses.append(
                        roll(active_action, num_rollouts - 1))

            return action

        initial = Action(STATIC, initial_action, [])

        return roll(initial, self.opt['dialog_rounds'])

In [21]:
from parlai.scripts.display_model import setup_args, DisplayModel
from parlai.core.params import ParlaiParser
from parlai.core.script import ParlaiScript, register_script
from parlai.core.agents import create_agent
from parlai.core.worlds import create_task

import random

def display_model(opt):
    random.seed(42)

    # Create model and assign it to the specified task
    agent = create_agent(opt)
    world = create_task(opt, agent)

    return agent, world

@register_script('display_model', aliases=['dm'])
class GetPretrainedAgent(ParlaiScript):
    @classmethod
    def setup_args(cls):
        return setup_args()

    def run(self):
        return display_model(self.opt)

agent, world = GetPretrainedAgent.main(
    task='empathetic_dialogues',
    model_file=os.path.join(working_dir, "from_pretrained/model"),
    num_examples=5,
    skip_generation=False,
)

03:43:18 | Overriding opt["model_file"] to /content/drive/MyDrive/Dev/ECEN689/ECEN689/from_pretrained/model (previously: from_pretrained/model)
03:43:18 | Overriding opt["skip_generation"] to False (previously: True)
03:43:18 | Using CUDA
03:43:18 | loading dictionary from /content/drive/MyDrive/Dev/ECEN689/ECEN689/from_pretrained/model.dict
03:43:18 | num words = 54944
03:43:20 | Total parameters: 87,508,992 (87,508,992 trainable)
03:43:20 | Loading existing model params from /content/drive/MyDrive/Dev/ECEN689/ECEN689/from_pretrained/model
03:43:23 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: valid


In [32]:
from parlai.core.agents import register_agent, Agent
from parlai.core.opt import Opt

# from parlai.agents.transformer import add_common_cmdline_args
import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Optional

@register_agent("RLTorchGeneratorAgent")
class RLTorchGeneratorAgent(agent.__class__):

    @classmethod
    def add_cmdline_args(
        cls, parser: ParlaiParser, partial_opt: Optional[Opt] = None
    ) -> ParlaiParser:
        """
        Add command-line arguments specifically for this agent.
        """
        agent = parser.add_argument_group('Transformer Arguments')
        # add_common_cmdline_args(agent)
        cls.dictionary_class().add_cmdline_args(parser, partial_opt=partial_opt)

        super().add_cmdline_args(parser, partial_opt=partial_opt)
        return agent

    @torch.no_grad()
    def sample_step(self, batch):
        """
        Sample a single batch of examples which will be used as 
        labels for the train step.
        """
        self.model.eval()

        _, preds, _ = self.model(
            *self._model_input(batch), ys=batch.label_vec, 
            use_probabilistic_decode=True)

        self.add_labels(batch, preds)

        return batch

    def add_labels(self, batch, label_vecs):
        """"""
        labels = [self._v2t(l) for l in label_vecs]
        ys, y_lens = padded_tensor(
            label_vecs, self.NULL_IDX, self.use_cuda)
            
        batch.labels=labels
        batch.label_vec=ys
        batch.label_lengths=y_lens

    def compute_log_prob(self, batch, track_metrics=True):
        """"""
        if batch.label_vec is None:
            raise ValueError('Cannot compute loss without a label.')

        model_output = self.model(
            *self._model_input(batch), ys=batch.label_vec)
        scores, preds, *_ = model_output
        score_view = scores.view(-1, scores.size(-1))
        log_prob = self.criterion(score_view, batch.label_vec.view(-1))

        notnull = batch.label_vec.ne(self.NULL_IDX)
        target_tokens = notnull.long().sum().item()
        correct = ((batch.label_vec == preds) * notnull).sum().item()

        if track_metrics:
            self.metrics['correct_tokens'] += correct
            self.metrics['nll_loss'] += log_prob.item()
            self.metrics['num_tokens'] += target_tokens

        log_prob /= target_tokens  # average loss per token

        return log_prob

    def train_step(self, batch):
        """
        Train on a single batch of examples.
        """
        try:
            batch = self.sample_step(batch)
            log_prob = self.compute_log_prob(batch)
            self.metrics['loss'] += log_prob.item()
            self.log_probs.append(log_prob)

        except RuntimeError as e:
            if 'out of memory' in str(e):
                print('| WARNING: ran out of memory, skipping batch. ')
                self.metrics['total_skipped_batches'] += 1
                # gradients are synced on backward, 
                # now this model is going to be
                # out of sync! catch up with the other workers
                self._init_cuda_buffer(8, 8, True)
            else:
                raise e
        
        return Output(
            text=batch.labels,
            text_vec=batch.label_vec,
            text_candidates=None)
        
    def observe(self, observation):
        """
        Calculates the reward and copies the model if provided
        additionally to the ``super().observe()``.
        """
        if observation.get('reward') is not None:
            reward = observation['reward']
            for log_prob in self.log_probs:
                loss = - log_prob * reward
                self.backward(loss)
            
            self.log_probs = []

            for parameter in self.model.parameters():  # pylint: disable=access-member-before-definition
                if parameter.grad is not None:
                    parameter.grad.data.clamp_(min=-5, max=5)
            
        if observation.get('model') is not None:
            # Deepcopied and frozen clone of the model
            self.model = observation['model']

        return super(RLTorchGeneratorAgent, self).observe(
            observation)
        
    def match_batch(self, batch_reply, valid_inds, output=None):
        """"""
        batch_reply = super(RLTorchGeneratorAgent, self).match_batch(
            batch_reply, valid_inds, output)

        if getattr(output, 'text_vec', None) is not None:
            for idx, val_idx in enumerate(valid_inds):
                batch_reply[val_idx]['text_vec'] = output.text_vec[idx]
        
        return batch_reply
        
    def build_model(self, *args, **kwargs):
        super(RLTorchGeneratorAgent, self).build_model(
            *args, **kwargs)
        assert model is not None, ('Model must be initialized first')
        model.forward = forward.__get__(model)
        
    # def build_model(self):
    #     model = MyModel


In [22]:
def get_agent_type(opt):
    model_file = opt.get("model_file")
    optfile = model_file + '.opt'
    if isfile(optfile):
        new_opt = Opt.load(optfile)
        print(new_opt)            

def create_agent(opt):
    """
    Creates a new class, that extends the provided 
    subclass of ``TorchGeneratorAgent`` with reinforcement
    learning functionality.
    """
    torch_generator_agent_subclass = get_agent_type(opt)

In [24]:
from parlai.scripts.train_model import setup_args, TrainLoop
from parlai.scripts.build_dict import build_dict

def get_opt():
    parser = setup_args()
    reinforce = parser.add_argument_group('Reinforce Arguments')
    reinforce.add_argument('--dialog_rounds',type=int, default=2) # Number of rollouts rounds for estimating the reward.
    reinforce.add_argument('--dialog_branches', type=int, default=5) # Branches of the active agent responses during rollout.
    reinforce.add_argument('--language_model_path',type=str,default=os.path.join(working_dir, "from_pretrained/model"))
    reinforce.add_argument('--reward_decay',type=float,default=0.9)
    reinforce.add_argument('--model_file',type=str, default=os.path.join(working_dir, "from_pretrained/model") )
    return parser.parse_args("")

class RLLoop(TrainLoop):
    def __init__(self, opt):
        signal.signal(signal.SIGINT, signal.default_int_handler)

        # training statistic checkpoint
        trainstats_suffix = ".trainstats"
        if opt.get('model_file') and isfile(opt['model_file'] + '.checkpoint'):
            trainstats_suffix = ".checkpoint.trainstats"
        
        self.agent = create_agent(opt)
        

opt = get_opt()

In [29]:
DisplayModel.main(
    task='empathetic_dialogues', model='RLTorchGeneratorAgent', 
    skip_generation=True,
)

03:46:39 | Using CUDA
03:46:39 | Total parameters: 3,511,200 (2,896,800 trainable)
03:46:39 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: valid
03:46:40 | Opt:
03:46:40 |     activation: relu
03:46:40 |     adafactor_eps: '(1e-30, 0.001)'
03:46:40 |     adam_eps: 1e-08
03:46:40 |     add_p1_after_newln: False
03:46:40 |     allow_missing_init_opts: False
03:46:40 |     attention_dropout: 0.0
03:46:40 |     batchsize: 1
03:46:40 |     beam_block_full_context: True
03:46:40 |     beam_block_list_filename: None
03:46:40 |     beam_block_ngram: -1
03:46:40 |     beam_context_block_ngram: -1
03:46:40 |     beam_delay: 30
03:46:40 |     beam_length_penalty: 0.65
03:46:40 |     beam_min_length: 1
03:46:40 |     beam_size: 1
03:46:40 |     betas: '(0.9, 0.999)'
03:46:40 |     bpe_add_prefix_space: None
03:46:40 |     bpe_debug: False
03:46:40 |     bpe_dropout: None
03:46:40 |     bpe_merge: None
03:46:40 |     bpe_vocab: None


In [46]:
opt.get("model_file")